This code needs SSIS package directory as input and generates convereted SSIS package 

In [69]:
import xml.etree.ElementTree as ET
import regex as re
import os
ET.register_namespace('DTS',"www.microsoft.com/SqlServer/Dts")
ET.register_namespace('SQLTask',"www.microsoft.com/sqlserver/dts/tasks/sqltask")

#directory='C:\\balaji\\sourcecode\\etl-MPCSEDW\\MPCS_ETL_SSIS\\SSIS\\'
directory='C:\\balaji\\2020\\teradata_migration_analysis\\pkgs\\test\\edit_ssis'

def Check_Delaercomp_Comm(fname):#check  for callidus connections and return file name   
    p = re.compile(r"\L<words>", words=['DealerComp', 'DEALERCOMP', 'CALLIDUS', 'Callidus','Subsidy','SUBSIDY'])
    n = re.compile(r"\L<words>", words=['None'])
    root = ET.parse(fname).getroot()
    callidus=0
    #print(root.tag)
    #print(root.attrib)
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManagers'):
        for y in x.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
            for z in y.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                for m in z.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
                     if  p.search(str(m.attrib.get(("{www.microsoft.com/SqlServer/Dts}ConnectionString")))):
                        callidus=1
                        break
    return callidus,fname.split('\\')[-1]                        
def get_mer_connname_ids(fname):    #get mercury connectio name and id 
    q = re.compile(r"\L<words>", words=['Mercury','MERCURY'])
    root = ET.parse(fname).getroot()
    conname_n_id={}
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManagers'):
        for y in x.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
                if y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) is not None:
                    if  q.search(y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))):
                        conname_n_id[y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))]=y.attrib.get(("{www.microsoft.com/SqlServer/Dts}DTSID"))
                        
                     
    return conname_n_id 

def get_Callidus_connname_ids(fname):    #get Callidus Conn name and id 
    q = re.compile(r"\L<words>", words=['Callidus','CALLIDUS'])
    root = ET.parse(fname).getroot()
    call_conname_n_id={}
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManagers'):
        for y in x.iter('{www.microsoft.com/SqlServer/Dts}ConnectionManager'):
                if y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) is not None:
                    if  q.search(y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))):
                        call_conname_n_id[y.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName"))]=y.attrib.get(("{www.microsoft.com/SqlServer/Dts}DTSID"))
                        
                     
    return call_conname_n_id 

def get_mercury_sqltaks(fname,cnam_id):  #get sql task list with mercury as connection  
    root = ET.parse(fname).getroot()
    mercury_sql_tasks=[]
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}Executable'):
         if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}Description"))=="Execute SQL Task":
                for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :
                       # print (sql.attrib)
                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            mercury_sql_tasks.append(x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")))
                            
                    
    return mercury_sql_tasks 

def update_pkg(fname,cnam_id,call_cnam_id,path,fname_only): #update xml file 
    print(fname_only)
    root = ET.parse(fname)
#     root = re.sub(r'&lt;','<',root)
#     root = re.sub(r'&gt;','>',root)
    for x in root.iter('{www.microsoft.com/SqlServer/Dts}Executable'):
        if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) in audit_end_tsks: # modify audit end task
             for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :                 

                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection']=list(call_cnam_id.values())[0]
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlStatementSource']="exec Create_AuditEvent ?,240,0,NULL,?,'Package Completed'"
                            for sql2 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                                               
                                if (sql2.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName')))=="System::PackageName":
                                    #print('s')
                                    sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName']="1"
                            for sql3 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                if (sql3.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) in delete_var_list:
                                    sql.remove(sql3)
        if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) in audit_start_tsks: #modify audit start task
                for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :                 

                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection']=list(call_cnam_id.values())[0]
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlStatementSource']="Exec Create_AuditEvent ?,150,0,NULL,?,'Package Started'"
                            for sql2 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                                               
                                if (sql2.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName')))=="System::PackageName":
                                    #print('s')
                                    sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName']="1"
                            for sql3 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                if (sql3.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) in delete_var_list :
                                    sql.remove(sql3)

                                    
        if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) in audit_error_tsks: #modify audit error task
                #print('err')
                for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :                 

                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection']=list(call_cnam_id.values())[0]
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlStatementSource']="EXEC Create_AuditEvent ?,  180, ?, ?, ?, ?"
                            for sql2 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                                               
                                if (sql2.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) not in delete_var_list +['User::ExtractDate']:
                                    val=int(sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName'])
                                    sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName']=str(val-1)
                            for sql3 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                if (sql3.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) in delete_var_list :
                                    sql.remove(sql3)
                        
        if x.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) in audit_rcount_tsks: #modify record count task
                #print('err')
                for y in x.iter('{www.microsoft.com/SqlServer/Dts}ObjectData'):
                    for sql in y.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlTaskData')  :                 

                        if (sql.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'))) in list(cnam_id.values()):
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection']=list(call_cnam_id.values())[0]
                            sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}SqlStatementSource']="EXEC Create_AuditEvent ?, 210, ?, NULL, ?,  'New Rows'"
                            for sql2 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                                               
                                if (sql2.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) not in delete_var_list_rcount +['User::ExtractDate']:
                                    val=int(sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName'])
                                    sql2.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterName']=str(val-1)
                            for sql3 in sql.iter('{www.microsoft.com/sqlserver/dts/tasks/sqltask}ParameterBinding'):
                                if (sql3.attrib.get(('{www.microsoft.com/sqlserver/dts/tasks/sqltask}DtsVariableName'))) in delete_var_list :
                                    sql.remove(sql3)                                
                            
                            #print(sql.attrib['{www.microsoft.com/sqlserver/dts/tasks/sqltask}Connection'])
#         if i.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")) in audit_start_tsks:
#             print (i.attrib.get(("{www.microsoft.com/SqlServer/Dts}ObjectName")))
#             print(i[0])
    print('Conversion complete for ', fname_only)
   
    root.write(os.path.join (directory+'_output',fname_only), xml_declaration=True,encoding='utf-8', method="xml")
            
                        
proc_calls={"call CreateAuditEvent(?,?,150,0,NULL,?,'Package Started')":"Exec Create_AuditEvent ?,150,0,NULL,?,'Package Started'",
"call CreateAuditEvent(?,?,240,?,NULL,?,'Package Completed')":"exec Create_AuditEvent ?,240,0,NULL,?,'Package Completed'",
"CALL CreateAuditEvent(?, ?, 180, ?, ?, ?, ?)":"EXEC Create_AuditEvent ?,  180, ?, ?, ?, ?",
 "CALL CreateAuditEvent(?, ?, 210, ?, NULL, ?,  'New Rows')"  : "EXEC Create_AuditEvent ?, 210, ?, NULL, ?,  'New Rows'"}   
pkg_sql_tasks={}
pkg_df_tasks={}
start_audit = re.compile(r"\L<words>", words=['Create Package Start Audi Event','Create Package Start Audit Event','Create Package Started Audit Event'])
for fname1 in os.listdir(directory):
    fname= os.path.join (directory,fname1)          
    CallidusPkg,pkg_name=Check_Delaercomp_Comm(fname)
    #print(CallidusPkg,pkg_name)
    cnam_id={}
    call_cnam_id={}
    if CallidusPkg==1:
        #print(fname)
        cnam_id=get_mer_connname_ids(fname)
        #print(cnam_id)
        call_cnam_id=get_Callidus_connname_ids(fname)
        #print(call_cnam_id)
        if len(cnam_id)>=1:
            mercury_sql_tasks_list=get_mercury_sqltaks(fname,cnam_id)
            #print(mercury_sql_tasks_list)
            audit_start_tsks=['Create Package Start Audi Event','Create Package Start Audit Event','Create Package Started Audit Event']
            audit_end_tsks=['Create Package Complete Audit Event','Create Package Completed Audit','Create Package Completed Audit Event','Create Package Complete Audit Event 1',' Create Package Complete Audit Event']
            delete_var_list=["User::ETLJobID","User::Row_Count"]
            delete_var_list_rcount=["User::ETLJobID"]
            audit_error_tsks=['Create OnError Audit Event']
            audit_rcount_tsks=[ 'Create RowCount Audit Event','Create RowCount Audit Event For Items','Create RowCount Audit Event For SOCRate','Create RowCount Audit Event record']
            start_audit_event=['yes' for i in mercury_sql_tasks_list if i in audit_start_tsks ]
            
            if len(start_audit_event) >0 :
                update_pkg(fname,cnam_id, call_cnam_id,directory,fname1)
            
           
 

m_Callidus_Subscriber_Dim.dtsx
Conversion complete for  m_Callidus_Subscriber_Dim.dtsx
m_SCRebate_Trans_load.dtsx
Conversion complete for  m_SCRebate_Trans_load.dtsx
PortInTransLoad.dtsx
Conversion complete for  PortInTransLoad.dtsx
Qpay_Payment_Fee_Summary.dtsx
Conversion complete for  Qpay_Payment_Fee_Summary.dtsx
